In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k #, AUC_at_k, ndcg_at_k

In [5]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items_weights = items_weights.sort_values(by=['weight'], ascending=False) # Сортируем по убыванию веса
    items_rec = items_weights.head(10) # выбираем для рекомендаций 10 товаров с самым большим весом
  
    items = np.array(items_rec['item_id'])
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [9]:
#%%time
# your_code
items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['log'] = np.log(items_weights['sales_value'])
log = np.array(items_weights['log'])
sum_ = log[np.isfinite(log)].sum()
items_weights['weight'] = items_weights['log'] / sum_
del items_weights['sales_value']
del items_weights['log']

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(5)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533765, 1082185, 6533889, 1029743, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 1029743, 6534166, 5569230, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6533765, 1029743, 995242, 916122, 1106523]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5569230, 6534178, 6533889, 6534166, 6533765]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[916122, 6533765, 5569230, 1106523, 6534178]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [137]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [138]:
# Исправим ошибку в датасете в признае actual
import re
act = result['actual']
actual_pars = []
for i in range(len(act)):
    tmp = re.findall(r'\d+', act[i])
    tmp1 = []
    for x in tmp:
        tmp1.append(int(x))
    actual_pars.append(tmp1)
result['actual_pars'] = actual_pars
del result['actual']
result.head(2)

,user_id,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,actual_pars
0,1,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[835476, 851057, 872021, 878302, 879948, 90963..."


In [149]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [154]:
# все равно что-то пошло не так.... везде 0
precision_at_k(result['popular_recommendation'], result['actual_pars'], k=5)

0.0

In [151]:
precision_at_k(result['itemitem'], result['actual_pars'], k=5)

0.0

In [152]:
precision_at_k(result['tfidf'], result['actual_pars'], k=5)

0.0

In [153]:
precision_at_k(result['own_purchases'], result['actual_pars'], k=5)

0.0

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code